# 1. 前準備 (csvの読み込みと紐づけ)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns #seabornない人はpip installしてね
import os
from datetime import datetime
import numpy as np
from pathlib import Path

# カレントディレクトリを.pyと合わせるために以下を実行
if Path.cwd().name == "notebook":
    os.chdir("..")

# 設定
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 500)
pd.set_option('display.max_columns', 500)

# 浮動小数点数を小数点以下3桁で表示するように設定
pd.set_option('display.float_format', '{:.3f}'.format)

In [ ]:
# Mac Matplotlibのデフォルトフォントをヒラギノ角ゴシックに設定
plt.rcParams['font.family'] = 'Hiragino Sans'

In [ ]:
# Windows MatplotlibのデフォルトフォントをMeiryoに設定
# plt.rcParams['font.family'] = 'Meiryo'

In [ ]:
# point_history.csvの読み込み
file_path_point_history = 'data/input/point_history_cleansing.csv'
df_point_history = pd.read_csv(file_path_point_history)

df_point_history = df_point_history.drop(columns=['total_amount'])    # 'total_amount'は全部N

In [ ]:
# user_info.csvの読み込み
file_path_user_info = 'data/input/user_info_cleansing.csv'
df_user_info = pd.read_csv(file_path_user_info)

In [ ]:
# user_infoにageとage_groupのカラムを追加
# ageの算出・追加
df_user_info['birth_day'] = pd.to_datetime(df_user_info['birth_day'], errors='coerce')    # birth_dayをdatetimeに変換
current_year = pd.Timestamp.now().year
df_user_info['age'] = current_year - df_user_info['birth_day'].dt.year

# 年代の算出・追加
bins = [0, 20, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ['0-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100']
df_user_info['age_group'] = pd.cut(df_user_info['age'], bins=bins, labels=labels, right=False)

In [ ]:
# point_history.csvとuser_info.csvをmerge (左内部結合)
df_merge = pd.merge(df_point_history, df_user_info, left_on='user_id', right_on='id', how='left')

In [ ]:
# マージしたコードの確認
# display(df_merge.head(100))
# df_merge["total_amount"].unique()
display(df_merge.describe())
# #print(len(df_merge["user_uid"].unique()))
# #print(df_merge.info(verbose=True, show_counts=True))
# print(df_merge['nickname'].isna().sum())

# 2. 分析

## 2-1. rankごとの平均年間持込量kgの算出

・rank_id毎のrecycle_amount_per_year平均の算出  
・rankが高いほど、年間持ち込み量が多いという仮説を証明する(当然？)  
・rank_idは、0:ブロンズ、1:シルバー、2:ゴールド、3:プラチナ  
・1か月に15kg持ち込み続けるとランクUP。ランクダウンは不明(1か月15kgを達成しないとダウン説、、？)  
※以下の手順を踏むと「rank毎の1回の持ち込み量平均」になってしまうので注意  
　　① amount_kgは1回の持ち込み量。これをrank_id毎にsumする  
　　② rank_idそれぞれの出現回数をsumする  
　　③ ①を②で割る  

In [ ]:
# まずそれぞれのカラムの基本統計量を確認しておく
print('recycle_amount_per_year')
print(df_merge['recycle_amount_per_year'].describe())
print('------------------')
print('rank_id')
print(df_merge['rank_id'].describe())
print('------------------')
print('rank_idのユニーク')
print(df_merge['rank_id'].unique())

※rankは3（プラチナ）が不在な点に注意

In [ ]:
# user_idの重複行を削除（非効率だと思いつつ、、、）
df_merge_duplicates_user_id = df_merge.drop_duplicates(subset='user_id')

ランク毎の持ち込みamountの平均 = df_merge.groupby('rank_id')['recycle_amount_per_year'].mean()
ランク毎の持ち込みamountの平均

### 得られた示唆

・ランクが高いほど、平均年間持ち込み量は多い  
　→「持ち込み量が多いから、ランクが上がった」という解釈の方が正しい？  
 ・rankのカラムのみ、user_infoに紐づけておくと、user_infoの情報量が増えてよいかも


## 2-2. ユーザ個人に着目して指針を得る 

### 2-2-1. トップユーザのamount_kg推移

・point、coin、rankが、ユーザの持ち込み量に影響を与えているのか確認  
・rankを維持したい人


In [ ]:
# 'user_id'の各値の出現回数を計算
user_id_counts = df_merge['user_id'].value_counts().reset_index()
user_id_counts.columns = ['user_id', 'count']

# 元のDataFrameにcountをマージ
df_merge = df_merge.merge(user_id_counts, on='user_id')

# 出現回数に基づいてソート（降順）
df_merge = df_merge.sort_values(by='count', ascending=False)

In [ ]:
# 重複を削除して、上位100名のcountをtableで俯瞰する
df_merged_duplicates_user_id = df_merge.drop_duplicates(subset='user_id')
top_100 = df_merged_duplicates_user_id.head(100)
top_100

In [ ]:
# top_100の年代別人数を表示
sns.histplot(data=top_100, x='age_group')

plt.title('Age Band Distribution')
plt.xlabel('Age Band')
plt.ylabel('Frequency')

plt.show()

In [ ]:
# # 特定の人のamount_kgの推移を確認
# # 特定の人のデータのみ抽出
# target_user_id = 1152
# df_merge_target = df_merge[df_merge['user_id'] == target_user_id]
# df_merge_target = df_merge_target.sort_values(by='use_date', ascending=True)

# df_merge_target['use_date'] = pd.to_datetime(df_merge_target['use_date'])

# # use_dateを月ごとに集約（データ数が多すぎて折れ線グラフが見づらい）
# df_merge_target_monthly = df_merge_target[['use_date', 'amount_kg']].resample('M', on='use_date').sum()

# # 集約されたデータで折れ線グラフを作成
# plt.figure(figsize=(10, 6))
# plt.plot(df_merge_target_monthly.index, df_merge_target_monthly['amount_kg'], marker='o')
# plt.title('Monthly Average of Amount (kg)')
# plt.xlabel('Month')
# plt.ylabel('Average Amount (kg)')

# # データをすべて使って折れ線グラフを作成
# plt.figure(figsize=(10, 6))
# plt.plot(df_merge_target_monthly['use_date'], df_merge_target_monthly['rank_id'], marker='o')
# plt.title('rank')
# plt.xlabel('date')
# plt.ylabel('rank')
# plt.show()

# # coinが付与され始めた時期のデータを表示
# filtered_df  = df_merge_target[df_merge_target['coin'] > 0]
# filtered_df.head()


### まとめ

・15kgを達成しても、すぐにランクが上がらない矛盾あり  
・15kgを未達でもすぐにランクが下がらない矛盾あり  
・アプリを利用していなくても履歴は残る。アプリを利用しているか否かは'coin'(付与コイン)で判断する  

## 2-3. rankシステムの効果測定

・rank_idのレコード

In [ ]:
# TODO: amount_kgの平均を出してしまっているので、user　rank_idごとのamount_kgの合計を出さないといけない？？
# Calculate the average recycling amount for each user and each rank
avg_amount_per_user_rank = df_merge.groupby(['user_id', 'rank_id'])['amount_kg'].mean().reset_index()
# avg_amount_per_user_rank = df_merge.groupby(['user_id', 'rank_id'])['amount_kg'].sum().reset_index()

# To compare the average recycling amount of different ranks for each user, we need to pivot the data
pivot_table = avg_amount_per_user_rank.pivot(index='user_id', columns='rank_id', values='amount_kg')

# Calculate the difference between each rank's average recycling amount for each user
# For simplicity, we'll calculate the difference between consecutive ranks (rank N and rank N-1)
rank_differences = pivot_table.diff(axis=1)

In [ ]:
rank_differences.head(100)

### 1,2,3ごとの平均値を出してみる

In [ ]:
average_difference_rank = rank_differences.mean(skipna=True)
average_difference_rank

### 1,2,3ごとの合計を出してみる

In [ ]:
# TODO: amount_kgの平均を出してしまっているので、user　rank_idごとのamount_kgの合計を出さないといけない？？
# Calculate the average recycling amount for each user and each rank
sum_amount_per_user_rank = df_merge.groupby(['user_id', 'rank_id'])['amount_kg'].sum().reset_index()
# avg_amount_per_user_rank = df_merge.groupby(['user_id', 'rank_id'])['amount_kg'].sum().reset_index()

# To compare the average recycling amount of different ranks for each user, we need to pivot the data
pivot_table = sum_amount_per_user_rank.pivot(index='user_id', columns='rank_id', values='amount_kg')

# Calculate the difference between each rank's average recycling amount for each user
# For simplicity, we'll calculate the difference between consecutive ranks (rank N and rank N-1)
rank_differences = pivot_table.diff(axis=1)

In [ ]:
sum_difference_rank = rank_differences.mean(skipna=True)
sum_difference_rank

## 2-4. 日付・時間・曜日ごとの古紙投入回数

・point_history.csvの既存のカラムでできる分析に取り組む  
・point_history_cleansing.csvはアプリユーザ限定。これを用いて分析。(元データは重すぎて扱えない)  
・投入量だと、一人で大量に投入したデータの影響が大きく、分析の目的がブレる。  

In [ ]:
# df_mergeのuse_date列から曜日情報を抽出
df_merge['use_date'] = pd.to_datetime(df_merge['use_date'])

In [ ]:
# amount列が0なのに行が残っているデータ:現状は削除する。（memo: なぜログが残るのか理由を聞きたい）
df_merge_drop_zero = df_merge[df_merge['amount'] != 0]

In [ ]:
# 曜日ごとのpoint_historyの回数をカウントしてグラフ化
df_merge_drop_zero['day_name'] = df_merge_drop_zero['use_date'].dt.day_name()

plt.bar(df_merge_drop_zero['day_name'].value_counts().index, df_merge_drop_zero['day_name'].value_counts().values)
plt.ylabel("古紙投入回数")
plt.show()

曜日順にする

In [ ]:
# 日付ごとの古紙投入回数
date_series = df_merge_drop_zero['use_date'].dt.date

date_series = date_series.value_counts().sort_index()
display(date_series.head(10))

plt.plot(date_series.index, date_series.values)
#plt.plot(date_series.index[900:950], date_series.values[900:950])  # monthlyレベルに拡大
plt.ylabel("古紙投入回数")
plt.xticks(rotation=45)
plt.show()

行政の回収日の影響を見るのであれば、仙台市ユーザーに絞ってデータ解析する必要がある  
もし古紙投入回数について規則性を見つけられれば、打ち手を考えやすい  
リサイクルポイントクラブ時代のデータがロスってないかが心配

In [ ]:
# 時間ごとの古紙投入回数
# df_mergeのuse_date列から時刻情報を抽出
time_series = df_merge_drop_zero['use_date'].dt.hour

time_series = time_series.value_counts().sort_index()
#display(time_series)

plt.plot(time_series.index, time_series.values)
#plt.plot(date_series.index[900:950], date_series.values[900:950])  # monthlyレベルに拡大
plt.ylabel("古紙投入回数")
plt.xticks(rotation=45)
plt.xticks(range(0, 24, 1))
plt.grid(True)
plt.show()

場所によってはRPSがいっぱいになるので、時間に応じてそこを調整できないか  
◎リサイクル充填率をリアルタイムで予測する  
　・いっぱいになっているタイミングの時の量を100%とする→ダメもとでも聞いてみる  
　・何日先まで予測できるといいのか  
　・充填率の予測にどれくらい需要があるのか？  

In [ ]:
print(df_merge_drop_zero['rps_opening_time'].unique())
print(df_merge_drop_zero['rps_closing_time'].unique())

### 店舗ごとにグループ化してリサイクル量の変化が止まっている箇所で充填率100%の量とする（仮説）

In [ ]:
df_merge_drop_zero

In [ ]:
#充填率を見るために、取引量が多い店舗、日付をまず見つける
df_merge_drop_zero['use_date2'] = df_merge_drop_zero['use_date'].dt.date

# shop_idとuse_dateでグループ化し、各グループのサイズを計算
grouped = df_merge_drop_zero.groupby(['shop_id', 'use_date2']).size()

# 最大のグループを見つける
max_group = grouped[grouped == grouped.max()]
max_group

In [ ]:
tempdf = df_merge_drop_zero[df_merge_drop_zero['shop_id'] == 1004]
#リサイクル量を時間ごとに累積していく
tempdf['use_hour'] = df_merge_drop_zero['use_date'].dt.hour
tempdf = tempdf[tempdf['use_date2'] == datetime(2023,2,18).date()]
tempdf = tempdf.sort_values('use_date')  # use_dateでソート
tempdf['use_date']
tempdf['cumulative_amount_kg'] = tempdf['amount_kg'].cumsum()  # amount_kgの累積値を計算し、新しい列に追加
plt.plot(tempdf['use_date'], tempdf['amount_kg'])
plt.ylabel("リサイクル量")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
plt.plot(tempdf['use_date'], tempdf['cumulative_amount_kg'])
plt.ylabel("リサイクル量")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

## 一日中稼働してる！！？？充填率100％にならないの？
#### どのくらい入るのか聞きたい
#### 一日一回回収じゃないのでは？18から22日でいっぱいになった？

In [ ]:
df_merge_drop_zero[df_merge_drop_zero['shop_id'] == 1004]['use_date']

## 2-3. 店舗ごとのポイント・コイン・リサイクル量・営業時間の可視化

### 2-3-1. スーパーの名前を一覧

In [ ]:
df_merge["super"].unique()

### 2-3-2. 店舗ごとのポイント・コイン・リサイクル量

#### 示唆
持込量が圧倒的に多いのはヨークベニマル。ただしヨークベニマルはリサイクルポイントシステムを導入していない。(というのがどういうことなのか確認)
持込量・獲得ポイントの平均値はどのスーパーでも大差なし。獲得コインはやや差がある様に見えるが、意味があるかは不明。

In [ ]:
df_merge_super_ptsum = df_merge.groupby('super')['point'].sum().sort_values(ascending=False)
df_merge_super_ptsum.plot(kind='bar')
plt.title('point x super (sum)')
plt.show()

In [ ]:
df_merge_super_ptsum = df_merge.groupby('super')['coin'].sum().sort_values(ascending=False)
df_merge_super_ptsum.plot(kind='bar')
plt.title('coin x super (sum)')
plt.show()

In [ ]:
df_merge_super_kgsum = df_merge.groupby('super')['amount_kg'].sum().sort_values(ascending=False)
df_merge_super_kgsum.plot(kind='bar')
plt.title('amount_kg x super (sum)')
plt.show()

In [ ]:
df_merge_super_ptmean = df_merge.groupby('super')['point'].mean().sort_values(ascending=False)
df_merge_super_ptmean.plot(kind='bar')
plt.title('point x super (mean)')
plt.show()

In [ ]:
df_merge_super_ptmean = df_merge.groupby('super')['coin'].mean().sort_values(ascending=False)
df_merge_super_ptmean.plot(kind='bar')
plt.title('coin x super (mean)')
plt.show()

In [ ]:
df_merge_super_kgmean = df_merge.groupby('super')['amount_kg'].mean().sort_values(ascending=False)
df_merge_super_kgmean.plot(kind='bar')
plt.title('amount kg x super (mean)')
plt.show()

### 2-3-3. 店舗 x 年代ごとの持込量合計(または平均)のクロス集計

In [ ]:
df_merge_pivot_sum = df_merge.pivot_table(index='super', columns='age_group', values='amount_kg', aggfunc='sum')
df_merge_pivot_sum

In [ ]:
df_merge_pivot_mean = df_merge.pivot_table(index='super', columns='age_group', values='amount_kg', aggfunc='mean')
df_merge_pivot_mean

### 2-3-4. 店舗ごとの店舗営業時間およびRPS稼働時間の可視化

### 示唆
#### 開店時間は9:00-9:30が多い。RPSも同様。
#### 閉店時間は21:00-22:00が多い。RPSは19時に閉まる店舗が多い。
#### RPSは9:00-19:00の稼働が多いが、店舗が長く営業していると、それに合わせてやや長く設定している。
#### 店舗営業時間に対するRPS稼働時間は、ほぼ100%の塊と、80%程度の塊が多い。 

In [ ]:
# 各列をdatetime型に変換する関数
def convert_to_datetime(df, column):
    df[column] = pd.to_datetime(df[column], format='%H:%M:%S', errors='coerce').dt.time

df_converted = df_merge.copy()
columns_to_convert = ['store_opening_time', 'store_closing_time', 'rps_opening_time', 'rps_closing_time']
for column in columns_to_convert:
    convert_to_datetime(df_converted, column)

In [ ]:
#分析に使用する列の追加((1)店舗営業時間、(2)RPS稼働時間、(3)店舗営業時間におけるRPS稼働時間の割合(2)/(1))
# 時間の差分を計算する関数
import datetime
def calculate_time_difference(time1, time2):
    if pd.isnull(time1) or pd.isnull(time2):
        return pd.NaT
    return datetime.datetime.combine(datetime.date.min, time2) - datetime.datetime.combine(datetime.date.min, time1)

df_converted['shop_business_hours'] = df_converted.apply(lambda row: calculate_time_difference(row['store_opening_time'], row['store_closing_time']), axis=1)
df_converted['rps_operating_hours'] = df_converted.apply(lambda row: calculate_time_difference(row['rps_opening_time'], row['rps_closing_time']), axis=1)
df_converted['rps_shop_hour_ratio'] = df_converted.apply(lambda row: row['rps_operating_hours'] / row['shop_business_hours'] if row['shop_business_hours'] != datetime.timedelta(0) else pd.NaT, axis=1)

print(df_converted.head())

In [ ]:
mode_store_open = df_merge.groupby('shop_name')['store_opening_time'].agg(pd.Series.mode)
store_open_counts = mode_store_open.value_counts()
store_open_counts.plot(kind='bar')
plt.title("開店時間集計")
plt.show()

In [ ]:
mode_rps_open = df_merge.groupby('shop_name')['rps_opening_time'].agg(pd.Series.mode)
rps_open_counts = mode_rps_open.value_counts()
rps_open_counts.plot(kind='bar')
plt.title("RPSオープン時間集計")
plt.show()

In [ ]:
mode_shop_close = df_merge.groupby('shop_name')['store_closing_time'].agg(pd.Series.mode)
shop_close_counts = mode_shop_close.value_counts()
shop_close_counts.plot(kind='bar')
plt.title("閉店時間集計")
plt.show()

In [ ]:
mode_rps_close = df_merge.groupby('shop_name')['rps_closing_time'].agg(pd.Series.mode)
rps_close_counts = mode_rps_close.value_counts()
rps_close_counts.plot(kind='bar')
plt.title("RPSクローズ時間集計")
plt.show()

In [ ]:
# timedelta を分数に変換する関数
def timedelta_to_minutes(td):
    return td.total_seconds() / 60

# timedelta 列を分数に変換
df_converted['shop_business_hours_mins'] = df_converted['shop_business_hours'].apply(timedelta_to_minutes)
df_converted['rps_operating_hours_mins'] = df_converted['rps_operating_hours'].apply(timedelta_to_minutes)

In [ ]:
def safe_mode(series):
    modes = series.mode()
    if len(modes) > 0:
        return modes.iloc[0]
    else:
        return np.nan

# 各店舗ごとに各列のモードを計算
shop_mode_values = df_converted.groupby('shop_name').agg({'shop_business_hours_mins': safe_mode,
                                                          'rps_operating_hours_mins': safe_mode,
                                                          'rps_shop_hour_ratio': safe_mode})


In [ ]:
# ヒストグラムを描画
plt.figure(figsize=(15, 5))

# shop_business_hours_mins のヒストグラム
plt.subplot(1, 3, 1)
#plt.hist(shop_mode_values['shop_business_hours_mins'].dropna(), bins=10, color='blue', alpha=0.7)
plt.hist(shop_mode_values['shop_business_hours_mins'].dropna(), bins=10, range=(480,1000), color='blue', alpha=0.7)
plt.title('①各店舗の営業時間')
plt.xlabel('Minutes')
plt.ylabel('Frequency')

# rps_operating_hours_mins のヒストグラム
plt.subplot(1, 3, 2)
plt.hist(shop_mode_values['rps_operating_hours_mins'].dropna(), bins=10, range=(480,1000), color='green', alpha=0.7)
plt.title('②各RPSの稼働時間')
plt.xlabel('Minutes')

# rps_shop_hour_ratio のヒストグラム
plt.subplot(1, 3, 3)
plt.hist(shop_mode_values['rps_shop_hour_ratio'].dropna(), bins=10, range=(0.6,1.1), color='red', alpha=0.7)
plt.title('②RPS稼働時間 / ①店舗営業時間')
plt.xlabel('Ratio')

plt.tight_layout()
plt.show()


In [ ]:
# 営業時間がマイナスの行を表示
negative_shop_business_hours = df_converted[df_converted['shop_business_hours_mins'] < 0]
print(len(negative_shop_business_hours))
negative_shop_business_hours

In [ ]:
# 営業時間が900分以上の行を表示
long_shop_business_hours = df_converted[df_converted['shop_business_hours_mins'] >900]
print(len(long_shop_business_hours))
long_shop_business_hours

In [ ]:
# RPS稼働が900分以上の行を表示
long_rps_operating_hours = df_converted[df_converted['rps_operating_hours_mins'] >900]
print(len(long_rps_operating_hours))
long_rps_operating_hours